### Ruben Abbou
## A Sparse Structured Matrix Class
This project contains a data structure which holds tridiagonal matrices with an edge such as
$$C = \begin{bmatrix} d_0 & b_0 & 0 & 0 & e_0 \\
a_0 & d_1 & b_1 & 0 & e_1 \\
0 & a_1 & d_2 & b_2 & e_2 \\
0 & 0 & a_2 & d_3 & b_3 \\
0 & 0 & 0 & a_3 & d_4\end{bmatrix}.$$
As well as a solver for equations of the form $C\vec{x} = \vec{r}$. The solver is an improved version of Gaussian Elimination for this specific problem and has $8n$ complexity.

In [1]:
import time
from numpy import array, zeros, sin, mean

In [2]:
class Edge_Tridiag:
    '''
    Object type to represent square edge-tridiagonal matrices.
    Contains a multiplication and copying methods as well as a printing routine,
    which does not take up memory.
    '''
    def __init__(self, d, a, b, e):
        self.d = d
        self.a = a
        self.b = b
        self.e = e
        self.n = len(d)
        
        # Check array dimensions
        assert len(self.a) == self.n-1
        assert len(self.b) == self.n-1
        assert len(self.e) == self.n-2

    def multiply(self, x):
        assert self.n == len(x)
        out = zeros(self.n)
        for i in range(self.n):
            if i-1 >= 0:
                out[i] += self.b[i-1]*x[i-1]
            if i+1 < self.n:
                out[i] += self.a[i]*x[i+1]
            if i < self.n - 2:
                out[i] += self.e[i]*x[-1]
            out[i] += self.d[i]*x[i]
        return array(out).reshape(-1,1)
    
    def copy(self):
        return Edge_Tridiag(array(self.d), array(self.a),
                            array(self.b), array(self.e))
    
    def __str__(self):
        out = '['
        for i in range(self.n):
            if i != 0: out += ' '
            out += '['
            for j in range(self.n):
                if abs(i-j) > 1:
                    if j == self.n-1 and i < self.n-2:
                        out += "{:.2f}".format(self.e[i])
                    else:
                        out += '0   '
                elif j == i: out += "{:.2f}".format(self.d[i])
                elif i == j-1: out += "{:.2f}".format(self.a[i])
                elif j == i-1: out += "{:.2f}".format(self.b[j])
                else: print(i, j, abs(i-j))
                if j < self.n - 1: out += ' '
            out += ']'
            if i != self.n-1 : out += '\n'
        return out + ']'

In [3]:
def forward_elim(C, r):
    A = C.copy()
    x = array(r)
    
    # Normalize
    A.a[0] /= A.d[0]
    A.e[0] /= A.d[0]
    x[0] /= A.d[0]
    A.d[0] = 1
    
    for i in range(1, A.n):
        A.d[i] -= A.b[i-1] * A.a[i-1]
        if i < A.n-2:
            A.e[i] -= A.b[i-1] * A.e[i-1]
            A.e[i] /= A.d[i] # normalize
            
        if i == A.n-2:
            A.a[i] -= A.b[i-1] * A.e[i-1]

        x[i] -= A.b[i-1] * x[i-1]
        
        # normalize
        x[i] /= A.d[i]
        if i < A.n-1:
            A.a[i] /= A.d[i]
        A.d[i] = 1
    return A, x

def back_substitution(C, r):
    x = zeros((C.n, 1))
    x[-1] = r[-1]

    for i in range(C.n-2, -1, -1):
        x[i] = r[i] - C.a[i] * x[i+1]
        if i < C.n-2:
            x[i] -= C.e[i] * x[-1]
    return x

def gauss_e_t_solve(C, r):
    '''
    Gaussian elimination algorithm to solve Cx=r tridiagonal matrices
    Inputs:
        - C: Edge_Tridiag object
        - r: array
    Outputs:
        - y: solution vector
    '''
    if len(r) != C.n:
        raise NameError('dimensions do not match')
    if any(C.d == 0):
        raise NameError('cannot have zeros along the diagonal')
    
    C, r = forward_elim(C, r)
    x = back_substitution(C, r)
    return x

In [4]:
n = 5
d = array([4 + 0.1 * k for k in range(n)])
a = array([1 + 0.01 * k**2 for k in range(n-1)])
b = array([1 - 0.01 - 0.03 * k for k in range(n-1)])
e = array([1 - 0.05 * k for k in range(n-2)])
C = Edge_Tridiag(d, a, b, e)
y = array([i for i in range(1, n+1)]).reshape(-1,1)
r = C.multiply(y)
print('r = ')
print(r)

x = gauss_e_t_solve(C, r)
print('\nGaussian Elimination solution for x: ')
print(x)

print('\nIf solution is right, x should be equal to y:')
print('{:.2f}%'.format(mean(abs(x-y)<1e-12)*100))

r = 
[[11.  ]
 [16.97]
 [23.18]
 [25.44]
 [25.6 ]]

Gaussian Elimination solution for x: 
[[1.]
 [2.]
 [3.]
 [4.]
 [5.]]

If solution is right, x should be equal to y:
100.00%


In [5]:
start = time.time()
n2 = 40000
d2 = array([4+0.1*sin(i/1000) for i in range(n2)])
a2 = array([0.2*sin(i/2000) for i in range(n2-1)])
b2 = array([0.3*sin(i/3000) for i in range(n2-1)])
e2 = array([0.4*sin(i/4000) for i in range(n2-2)])
C2 = Edge_Tridiag(d2, a2, b2, e2)
y2 = array([i for i in range(1, n2+1)]).reshape(-1,1)
r2 = C2.multiply(y2)
x2 = gauss_e_t_solve(C2, r2)

print('\nIf solution is right, x should be equal to y:')
print('{:.2f}%'.format(mean(abs(x2-y2) < 1e-10)*100))

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("\ntime to run Gaussian Elimination: {:0>2} min {:05.2f} sec".format(int(minutes),seconds))


If solution is right, x should be equal to y:
100.00%

time to run Gaussian Elimination: 00 min 01.64 sec


As desired, $x=y$.